Trabalhando com banco de dados

In [2]:
import os
from dotenv import load_dotenv
import psycopg2
from langchain.llms import OpenAI

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

def conectar_db():
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host='localhost',
            port=os.getenv("DB_PORT")
        )
        print("Conexão ao banco de dados PostgreSQL bem-sucedida!")
        return conn
    except psycopg2.OperationalError as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        raise

def obter_dados_acoes(pergunta):
    conn = conectar_db()
    cur = conn.cursor()
    query = """
    SELECT s.ticker, s.name, sp.date, sp.open, sp.high, sp.low, sp.close, sp.volume
    FROM stocks s
    JOIN stock_prices sp ON s.id = sp.stock_id
    WHERE s.name ILIKE %s OR s.ticker ILIKE %s;
    """
    cur.execute(query, (f"%{pergunta}%", f"%{pergunta}%"))
    resultados = cur.fetchall()
    cur.close()
    conn.close()
    return resultados

llm = OpenAI(temperature=0.6, api_key=os.getenv("OPENAI_API_KEY"))

def analisar_dados_acoes(dados_acoes): 
 
    prompt = "Analise os seguintes dados de ações e forneça insights detalhados:\n"
    
    for dado in dados_acoes:
        prompt += (f"Ticker: {dado[0]}, Nome: {dado[1]}, Data: {dado[2]}, "
                   f"Abertura: {dado[3]}, Máxima: {dado[4]}, Mínima: {dado[5]}, "
                   f"Fechamento: {dado[6]}, Volume: {dado[7]}\n")
 
    prompt += "\nPor favor, forneça um resumo e quaisquer tendências ou insights notáveis."
    
    insights = llm(prompt)
    return insights

def responder_pergunta(pergunta):
    dados_acoes = obter_dados_acoes(pergunta)
    if dados_acoes:
        resposta = f"Dados encontrados para '{pergunta}':\n"
        for dado in dados_acoes:
            resposta += (f"Ticker: {dado[0]}, Nome: {dado[1]}, Data: {dado[2]}, "
                         f"Abertura: {dado[3]}, Máxima: {dado[4]}, Mínima: {dado[5]}, "
                         f"Fechamento: {dado[6]}, Volume: {dado[7]}\n")
        resposta += "\nInsights da OpenAI:\n"
        resposta += analisar_dados_acoes(dados_acoes)
    else:
        resposta = f"Nenhum dado encontrado para '{pergunta}'."
    return resposta

def main():
    pergunta = input("Sobre qual ação você deseja saber informações? ")
    resposta = responder_pergunta(pergunta)
    print(resposta)

if __name__ == "__main__":
    main()
